In [10]:
import gzip, ast, json, pandas as pd

data = []
with gzip.open('steam_games.json.gz', 'rb') as f:
    for line in f:
        data.append(json.loads(line))
steam_games = pd.DataFrame(data)
f.close()


data = []
with gzip.open('user_reviews.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))
user_reviews = pd.DataFrame(data)
f.close()




In [11]:
data = []
with gzip.open('users_items.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))
user_items = pd.DataFrame(data)
f.close()

CREAR COLUMNA user_reviews.sentiment_analysis 
1. reviasamos el Dataframe user_reviews con( .info() y .head()) y preparamos la columna user_reviews.reviews para desanidarla pero la preparamos para que este formada por objetos 
.json 
2. procedemos crear la funcion para consegir user_reviews.sentiment_analysis 
3. eliminamos nulos 
4. creamos archivo user_reviews.csv

In [12]:
user_reviews.info() # mirando DataFrame observamos 3 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [13]:
user_reviews.head() # miramos los tres columnas y las 5 primeras filas del DataFrame 

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [14]:
user_reviews["reviews"] # mirando el pricipio y final de  columna user_reviews.reviews
# observamos que clave y valor tienen comillas simples  

0        [{'funny': '', 'posted': 'Posted November 5, 2...
1        [{'funny': '', 'posted': 'Posted June 24, 2014...
2        [{'funny': '', 'posted': 'Posted February 3.',...
3        [{'funny': '', 'posted': 'Posted October 14, 2...
4        [{'funny': '3 people found this review funny',...
                               ...                        
25794    [{'funny': '', 'posted': 'Posted May 31.', 'la...
25795    [{'funny': '', 'posted': 'Posted June 17.', 'l...
25796    [{'funny': '1 person found this review funny',...
25797    [{'funny': '', 'posted': 'Posted July 21.', 'l...
25798    [{'funny': '1 person found this review funny',...
Name: reviews, Length: 25799, dtype: object

Creamos  funcion para corregir columna user_reviews.reviews

In [15]:
import json

def corregir_reviews_a_json(df):
    def corregir_celda(celda):
        if isinstance(celda, str):
            try:
                # Intentar cargar la celda como JSON
                json_data = json.loads(celda)
                return json_data
            except json.JSONDecodeError:
                # Si hay un error al cargar como JSON, devolver la celda sin cambios
                return celda
        elif isinstance(celda, list):
            # Si la celda es una lista, convertirla a una cadena JSON
            return json.dumps(celda)
        else:
            # Si la celda no es ni una cadena ni una lista, devolverla sin cambios
            return celda

    # Aplicar la función a cada celda de la columna 'reviews'
    df['reviews'] = df['reviews'].apply(corregir_celda)
    return df

# Llamar a la función con tu DataFrame
user_reviews = corregir_reviews_a_json(user_reviews)


In [16]:
user_reviews["reviews"] # mirando columna reviews y vemos que ya si parece un json 

0        [{"funny": "", "posted": "Posted November 5, 2...
1        [{"funny": "", "posted": "Posted June 24, 2014...
2        [{"funny": "", "posted": "Posted February 3.",...
3        [{"funny": "", "posted": "Posted October 14, 2...
4        [{"funny": "3 people found this review funny",...
                               ...                        
25794    [{"funny": "", "posted": "Posted May 31.", "la...
25795    [{"funny": "", "posted": "Posted June 17.", "l...
25796    [{"funny": "1 person found this review funny",...
25797    [{"funny": "", "posted": "Posted July 21.", "l...
25798    [{"funny": "1 person found this review funny",...
Name: reviews, Length: 25799, dtype: object

procedemos a desanidar la columna user_reviews.reviews 

In [18]:
import pandas as pd
import json

def desanidar_reviews(df):
    # Crear listas vacías para cada nueva columna que queremos crear
    columnas_nuevas = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']
    for columna in columnas_nuevas:
        df[columna] = None
    
    # Iterar sobre cada fila del DataFrame
    for indice, fila in df.iterrows():
        # Desanidar los datos JSON de la columna 'reviews' en una lista
        datos_json = json.loads(fila['reviews'])
        
        # Verificar si los datos son una lista y si hay al menos un elemento
        if isinstance(datos_json, list) and len(datos_json) > 0:
            # Tomar el primer elemento de la lista como los datos desanidados
            elemento = datos_json[0]
            # Asignar los valores desanidados a las nuevas columnas
            for columna in columnas_nuevas:
                df.at[indice, columna] = elemento.get(columna, None)
    
    # Eliminar la columna 'reviews' original
    df.drop(columns=['reviews'], inplace=True)
    
    return df

# Llamar a la función con tu DataFrame
user_reviews = desanidar_reviews(user_reviews)


In [20]:
user_reviews.info()# Observamos que despues de desanidar ha sido reemplazada la columna reviews
#por 6 columnas ( 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25799 non-null  object
 1   user_url     25799 non-null  object
 2   funny        25771 non-null  object
 3   posted       25771 non-null  object
 4   last_edited  25771 non-null  object
 5   item_id      25771 non-null  object
 6   helpful      25771 non-null  object
 7   recommend    25771 non-null  object
 8   review       25771 non-null  object
dtypes: object(9)
memory usage: 1.8+ MB


In [24]:
# Ver tanto los valores nulos como los no nulos simultáneamente
print("Valores nulos :")
print(user_reviews.isnull().sum())
print(user_reviews.isnull().sum().sum())
print("\nValores no nulos:")
print(user_reviews.notnull().sum())
print(user_reviews.notnull().sum().sum())

Valores nulos :
user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64
196

Valores no nulos:
user_id        25799
user_url       25799
funny          25771
posted         25771
last_edited    25771
item_id        25771
helpful        25771
recommend      25771
review         25771
dtype: int64
231995


In [25]:
user_reviews.isnull().sum().sum() # total nulos del DataFrame user_reviews
user_reviews =  user_reviews.dropna() #  user_reviews sin nulos

In [30]:
# Volvemos a mirar tanto los valores nulos como los no nulos simultáneamente y vemos que ya no hay nulos
print("Valores nulos :")
print(user_reviews.isnull().sum())
print(user_reviews.isnull().sum().sum())
print("\nValores no nulos:")
print(user_reviews.notnull().sum())
print(user_reviews.notnull().sum().sum())

Valores nulos :
user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64
0

Valores no nulos:
user_id        25771
user_url       25771
funny          25771
posted         25771
last_edited    25771
item_id        25771
helpful        25771
recommend      25771
review         25771
dtype: int64
231939


CREAMOS ARCHIVO csv para DataFrame **user_reviews**

In [32]:
# Guardar el DataFrame user_reviews en un archivo CSV
user_reviews.to_csv('user_reviews.csv', index=False)

CREAMOS COLUMNA 'sentiment_analysis' y creamos un csv

In [33]:
import pandas as pd
from textblob import TextBlob

# Función para análisis de sentimiento
def analizar_sentimiento(reseña):
    if pd.isnull(reseña):
        return 1  # Valor neutral si la reseña está ausente
    else:
        analysis = TextBlob(reseña)
        polarity = analysis.sentiment.polarity
        if polarity < -0.1:
            return 0  # Sentimiento negativo
        elif polarity > 0.1:
            return 2  # Sentimiento positivo
        else:
            return 1  # Sentimiento neutral

# Cargar el DataFrame user_reviews
user_reviews = pd.read_csv('user_reviews.csv')

# Aplicar la función a la columna 'review' para crear la columna 'sentiment_analysis'
user_reviews['sentiment_analysis'] = user_reviews['review'].apply(analizar_sentimiento)

# Eliminar la columna 'review' original
user_reviews.drop(columns=['review'], inplace=True)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
user_reviews.to_csv('user_reviews_con_sentimiento.csv', index=False)


In [1]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
user_reviews_con_sentimiento = pd.read_csv('user_reviews_con_sentimiento.csv')

# Mostrar las primeras filas del DataFrame
print(user_reviews_con_sentimiento.head())


             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

                              funny                    posted last_edited  \
0                               NaN  Posted November 5, 2011.         NaN   
1                               NaN     Posted June 24, 2014.         NaN   
2                               NaN        Posted February 3.         NaN   
3                               NaN  Posted October 14, 2013.         NaN   
4  3 people found this review funny    Posted April 15, 2014.         NaN   

   item_id                                          helpful  recommend  \
0     1250  

In [2]:
user_reviews_con_sentimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25771 entries, 0 to 25770
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25771 non-null  object
 1   user_url            25771 non-null  object
 2   funny               4204 non-null   object
 3   posted              25771 non-null  object
 4   last_edited         2027 non-null   object
 5   item_id             25771 non-null  int64 
 6   helpful             25771 non-null  object
 7   recommend           25771 non-null  bool  
 8   sentiment_analysis  25771 non-null  int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 1.6+ MB


In [3]:
# Visualizar la columna 'sentiment_analysis' del DataFrame
columna_sentimiento = user_reviews_con_sentimiento['sentiment_analysis']
print(columna_sentimiento)


0        2
1        2
2        2
3        2
4        1
        ..
25766    2
25767    1
25768    1
25769    2
25770    1
Name: sentiment_analysis, Length: 25771, dtype: int64


In [4]:
# Visualizar las primeras 20 filas de la columna 'sentiment_analysis'
primeras_20_filas_sentimiento = user_reviews_con_sentimiento['sentiment_analysis'].head(20)
print(primeras_20_filas_sentimiento)


0     2
1     2
2     2
3     2
4     1
5     0
6     1
7     1
8     0
9     2
10    1
11    1
12    1
13    1
14    0
15    2
16    0
17    2
18    1
19    2
Name: sentiment_analysis, dtype: int64


SEGUIMOS CON  EL PROCESAMIENT DE LOS DataFrame 'user_items' y  'steam_games'